In [ ]:
%pwd

'C:\\ML Projects\\Facial_Emotion_Recognition_End2End'

In [ ]:
import os
from pathlib import Path
from dataclasses import dataclass  

In [ ]:
os.chdir('C:\ML Projects\Facial_Emotion_Recognition_End2End')

In [ ]:
%pwd

'C:\\ML Projects\\Facial_Emotion_Recognition_End2End'

In [ ]:
@dataclass(frozen=True)
class PrepareBaseModel_Arguments:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int
    params_weights: str
    params_include_top: bool


In [ ]:
from src.Facial_Emotion_Recognition.constant import *
from src.Facial_Emotion_Recognition.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManager:           #it will prepare or extract the arguments 

    def __init__(                      
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH
            ):

            
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModel_Arguments:
        
        config=self.config.prepare_base_model

        create_directories(config.root_dir)

        prepare_base_model_config= PrepareBaseModel_Arguments(
             base_model_path=Path(config.base_model_path),
             root_dir=Path(config.root_dir),
             updated_base_model_path=Path(config.updated_base_model_path),
             params_classes=self.params.CLASSES,
             params_image_size=self.params.IAMAGE_SIZE,
             params_include_top=self.params.INCLUDE_TOP,
             params_weights=self.params.WEIGHTS,
             params_learning_rate=self.params.LEARNING_RATE
                                                    )      

        return prepare_base_model_config    
                 

In [ ]:
import os
from zipfile import ZipFile
from urllib.request import Request
import tensorflow as tf

In [ ]:
class PrepareBaseModel:

    def __init__(self, config: PrepareBaseModel_Arguments):
        self.config=config

    def save_model( path: Path, model: tf.keras.Model):
        model.save(path)

    def get_base_model(self):

        self.model=tf.keras.applications.vgg16.VGG16(
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        
        if freeze_all :
            for layer in model.layers:
                model.trainable = False
        
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            activation='softmax',
            units=classes
        )(flatten_in)

        full_model=tf.keras.models.Model(

            inputs=model.input,
            output=prediction
            
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentrophy',
            metrics=['accuracy']
        )

        full_model.summary()

        return full_model
    

    @staticmethod
    def updated_base_model(self):

        self.full_model=self.prepare_full_model(

            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.learning_rate
        )




In [ ]:
try:
    config=ConfigurationManager()
    model_prepare=config.get_prepare_base_model_config()
    model=PrepareBaseModel(model_prepare)
    model.get_base_model()
    model.updated_base_model()

except Exception as e:
    raise e

[2023-10-14 15:35:17,474: INFO: common: yaml file : config\config.yaml loaded successfully]
[2023-10-14 15:35:17,484: INFO: common: yaml file : params.yaml loaded successfully]
[2023-10-14 15:35:17,488: INFO: common: created directories at :artifacts]
[2023-10-14 15:35:17,492: INFO: common: created directories at :a]
[2023-10-14 15:35:17,497: INFO: common: created directories at :r]
[2023-10-14 15:35:17,501: INFO: common: created directories at :t]
[2023-10-14 15:35:17,505: INFO: common: created directories at :i]
[2023-10-14 15:35:17,510: INFO: common: created directories at :f]
[2023-10-14 15:35:17,514: INFO: common: created directories at :a]
[2023-10-14 15:35:17,517: INFO: common: created directories at :c]
[2023-10-14 15:35:17,521: INFO: common: created directories at :t]
[2023-10-14 15:35:17,525: INFO: common: created directories at :s]
[2023-10-14 15:35:17,529: INFO: common: created directories at :/]
[2023-10-14 15:35:17,532: INFO: common: created directories at :p]
[2023-10-14

BoxKeyError: "'ConfigBox' object has no attribute 'base_model_path'"